# How to copy files between accounts 
演示在Linux 下 ， 账号间拷贝文件的方式


为什么无法直接用cp 复制其他用户账号下的文件呢 ？ 因为没有权限：

In [1]:
ls -ld ~/tmp

drwxr-xr-x 14 nscc-gz_jiangli nscc-gz 4096 May 20 15:35 /HOME/nscc-gz_jiangli/tmp


In [2]:
ls -ld ~

drwx------ 89 nscc-gz_jiangli nscc-gz_jiangli 12288 May 22 12:51 /HOME/nscc-gz_jiangli


In [3]:
ls -ld /HOME

drwxr-x--x 1800 root root 106496 May 23 14:05 /HOME


In [4]:
ls /HOME

ls: cannot open directory /HOME: Permission denied


In [5]:
cd /HOME/nscc_ts_1

bash: cd: /HOME/nscc_ts_1: Permission denied


In [6]:
cd /HOME/intel

In [8]:
ls -ld .

drwxr-xr-x 19 root root 4096 Jun  8  2015 .


从前面的演示中，我们可以看到，文件能否访问是受权限控制的。需要你有此文件的 r 权限（可读） ， 以及其所有父文件夹的可执行（访问）权限（x权限）。

如果需要在不同的账号下共享文件，可以考虑以下几种方式 ：

## 1. scp 命令  
用 scp 命令 和  rsync 命令 在账号间进行账号拷贝是最好的方案之一。不过需要您有两个账号的key 文件。
这两个命令都是通过 ssh 通道来进行拷贝 ， 命令用法也和 ssh 很像 。

In [9]:
scp --help 

usage: scp [-1246BCpqrv] [-c cipher] [-F ssh_config] [-i identity_file]
           [-l limit] [-o ssh_option] [-P port] [-S program]
           [[user@]host1:]file1 ... [[user@]host2:]file2


In [11]:
ssh -i ~/keys/id_rsa.ky nscc-gz_kangyouzhong@ln3-gn0   whoami

nscc-gz_kangyouzhong


可以看到，通过这个KEY 文件可以成功访问 KY 的账号，那么，scp 命令也没问题。

In [12]:
scp -i ~/keys/id_rsa.ky  ~/file.lj  nscc-gz_kangyouzhong@ln3-gn0:~/.

file.lj                                       100%    0     0.0KB/s   00:00    


可以看到文件 ~/file.lj 顺利从我的账号传到了 ky 的账号 

In [13]:
scp -i ~/keys/id_rsa.ky    nscc-gz_kangyouzhong@ln3-gn0:~/file.ky  ~/.

file.ky                                       100%    0     0.0KB/s   00:00    


同样的办法也可以将数据从ky 的账号拷贝到我的账号下。

但是， SCP 命令也有些问题：

* 不支持断点续传

* 对文件夹的传输支持不够好，可能会出现BUG 。

那么替代的解决办法是： rsync 命令 。 当然，对于临时的需求，这种方式简单易用，挺不错的。

## 2. rsync 命令 

In [15]:
rsync --help

rsync  version 3.0.6  protocol version 30
Copyright (C) 1996-2009 by Andrew Tridgell, Wayne Davison, and others.
Web site: http://rsync.samba.org/
Capabilities:
    64-bit files, 64-bit inums, 64-bit timestamps, 64-bit long ints,
    socketpairs, hardlinks, symlinks, IPv6, batchfiles, inplace,
    append, ACLs, xattrs, iconv, symtimes

rsync comes with ABSOLUTELY NO WARRANTY.  This is free software, and you
are welcome to redistribute it under certain conditions.  See the GNU
General Public Licence for details.

rsync is a file transfer program capable of efficient remote update
via a fast differencing algorithm.

Usage: rsync [OPTION]... SRC [SRC]... DEST
  or   rsync [OPTION]... SRC [SRC]... [USER@]HOST:DEST
  or   rsync [OPTION]... SRC [SRC]... [USER@]HOST::DEST
  or   rsync [OPTION]... SRC [SRC]... rsync://[USER@]HOST[:PORT]/DEST
  or   rsync [OPTION]... [USER@]HOST:SRC [DEST]
  or   rsync [OPTION]... [USER@]HOST::SRC [DEST]
  or   rsync [OPTION]... rsync://[USER@]HOST[:PORT]/SRC [

rsync 是复杂而强大的工具。不过一般来说，下面这种用法可以解决大多数问题：

In [16]:
rsync -e "ssh -i /HOME/nscc-gz_jiangli/keys/id_rsa.ky"   ~/file.lj  nscc-gz_kangyouzhong@ln3-gn0:~/.   

在 -e 里配置建立 ssh 连接需要的特殊参数，这里只是指定了 key 文件，有时候，你可能还需要指定其他的，如用 -p 指定端口。

## 3. 其他临时共享办法

In [19]:
cp ~/file.lj /tmp/.

In [20]:
chmod 777 /tmp/file.lj

如上例所示，将文件拷贝到一个公共文件夹下，并将权限放开，是一个比较简易的文件共享方式。

不过您需要明确这种做法带来的问题：

文件人人可以获取。

此共享文件夹可能会被清理。

如果此共享文件夹是在内存里（如tmp），需要在同一结点才能访问。并且如果文件太大会使系统出现故障。